In [1]:
import pandas as pd 
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold,cross_val_score
import warnings 
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

In [5]:
y = np.log1p(train['SalePrice'])
train.drop(['SalePrice'],axis = 1, inplace = True)

In [7]:
combined = pd.concat([train,test],axis = 0)

In [9]:
for col in combined.columns :
    if combined[col].dtype == 'object':
        combined[col] = combined[col].fillna('None')
    else :
        combined[col] = combined[col].fillna(combined[col].median())

In [11]:
for col in combined.select_dtypes(include='object').columns:
    le = LabelEncoder()
    combined[col] = le.fit_transform(combined[col])


In [13]:
X = combined.iloc[:len(y), :]
X_test = combined.iloc[len(y):, :]


In [15]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
cv = KFold(n_splits=5, shuffle=True, random_state=42)
rmse = np.sqrt(-cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=cv)).mean()
print(f"CV RMSE (log RMSE): {rmse:.4f}")


CV RMSE (log RMSE): 0.1450


In [17]:
model.fit(X, y)
preds = model.predict(X_test)
final_preds = np.expm1(preds)

In [23]:
submission['SalePrice'] = final_preds
submission.to_csv('house_price_submission.csv', index=False)
print("Submission file created: house_price_submission.csv")

Submission file created: house_price_submission.csv
